###**CHART** **PIPELINE**

Montaggio del drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installazione

In [ ]:
!pip install numpy
!pip install torch
!pip install cython
!pip install opencv-python
!pip install h5py
!pip install pytesseract
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pycocotools
!pip install easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,731 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

Import

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch
import cv2
from google.colab.patches import cv2_imshow
import pytesseract
import pandas as pd
import os
import re
import numpy as np
import pickle
import shutil

Inizializzazioni delle variabili

Definizione delle funzioni

In [ ]:
def load_object_from_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)



Run di CapFigX

In [ ]:
def runCapFigX(figCap_path,input_file_path,output_file_path):
  !python {figCap_path} --input_path={input_file_path} --output_path={output_file_path}

Copia output CapFigX nell'input di chartOcr

In [ ]:

import os

def copyCaptoChartOCR(output_file_path,inputChartOCR_path):
  estensioni_immagini=['.jpg']
  if os.path.exists(output_file_path):
          lista_elementi = os.listdir(output_file_path)
          cartelle = [elemento for elemento in lista_elementi if os.path.isdir(os.path.join(output_file_path, elemento))    ]
  else:
          print("La cartella {} non esiste.".format(output_file_path))
          #exit(1)
  cartelle= [c for c in cartelle if c!="xpdf"]
  dizionario_pdf_img=dict()
  for c in cartelle :
    lista_immagini = [nome for nome in os.listdir(os.path.join(output_file_path,c)) if os.path.splitext(nome)[1].lower() in estensioni_immagini ]
    if len(lista_immagini)>0:
      dizionario_pdf_img[c]=lista_immagini

  for d in dizionario_pdf_img.keys():
    percorso_cartella_pdf=os.path.join(inputChartOCR_path,d)
    os.makedirs(percorso_cartella_pdf, exist_ok=True)
    for im in dizionario_pdf_img[d]:
      shutil.copy(  os.path.join(output_file_path,d,im)  , percorso_cartella_pdf)

  return dizionario_pdf_img

Run di ChartOCR

In [ ]:
def runChartOCR(inputChartOCR_path,outputChartOCR_path):
  os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule/models/py_utils/_cpools")
  !python setup.py build_ext --inplace
  os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule/external")
  !make
  os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule")
  !python "test_pipeline_new.py" --image_dir {inputChartOCR_path} --output_dir {outputChartOCR_path}

Salvataggio sottoforma di file excel dell'output di chartOCR

In [ ]:
def saving(outputChartOCR_path,output_final_path, only_img=False):

  diz_df = dict()

  if only_img==False:
    if os.path.exists(outputChartOCR_path):
        lista_nomi_pdf = os.listdir(outputChartOCR_path)
        for nome_pdf in lista_nomi_pdf:
          diz_df[nome_pdf] =[]
          lista_nomi_img=os.listdir(os.path.join(outputChartOCR_path,nome_pdf))
          for nome_img in lista_nomi_img:
            if os.path.exists(os.path.join(outputChartOCR_path,nome_pdf,nome_img,"result.jpg")):
              labelled_img = os.path.join(outputChartOCR_path,nome_pdf,nome_img,"result.jpg")

            if os.path.exists(os.path.join(outputChartOCR_path,nome_pdf,nome_img,"chart_object.pkl")):
              pkl_chart = os.path.join(outputChartOCR_path,nome_pdf,nome_img,"chart_object.pkl")
              chart = load_object_from_file(pkl_chart)
              print("SI PKL")
              #creiamo dizionaro di df cosi li salvo su excel dopo
              diz_df[nome_pdf].append(chart.df_table)
              print("SI PKL")

            else:
              print("NO PKL")
  elif only_img:
    if os.path.exists(outputChartOCR_path):
        lista_nomi_pdf = os.listdir(outputChartOCR_path)
        for nome_pdf in lista_nomi_pdf:
          diz_df[nome_pdf] =[]
          lista_nomi_img=os.listdir(os.path.join(outputChartOCR_path))
          for nome_img in lista_nomi_img:
            if os.path.exists(os.path.join(outputChartOCR_path,nome_img,"result.jpg")):
              labelled_img = os.path.join(outputChartOCR_path,nome_img,"result.jpg")

            if os.path.exists(os.path.join(outputChartOCR_path,nome_img,"chart_object.pkl")):
              pkl_chart = os.path.join(outputChartOCR_path,nome_img,"chart_object.pkl")
              chart = load_object_from_file(pkl_chart)
              print("SI PKL")
              #creiamo dizionaro di df cosi li salvo su excel dopo
              diz_df[nome_pdf].append(chart.df_table)
              print("SI PKL")

            else:
              print("NO PKL")

  for k in diz_df.keys():
    cartella_pdf=os.path.join(output_final_path,k)
    if not os.path.exists(cartella_pdf):
          os.makedirs(cartella_pdf)

    for i,df in enumerate(diz_df[k]):
      if df is None:
        print(f"{k}_{i} Dataframe Vuoto ")
      if df is not None:
        df.to_excel(os.path.join(cartella_pdf,"dataframes.xlsx"), sheet_name=f'img_{i+1}')

In [ ]:
def main_chart( input_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/input_file",output_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/output_final" ,output_list='a'):
  figCap_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/PDFigCapX/code/FigCap.py"
  input_file_path= input_path
  if not os.path.exists(os.path.join(output_path,"intermediate_dir")):
            # Se non esiste, creala
            os.makedirs(os.path.join(output_path,"intermediate_dir"))
  #output_file_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/output_file/"
  output_file_path=os.path.join(output_path,"intermediate_dir")
  if not os.path.exists(os.path.join(output_path,"intermediate_chartocrinput_dir")):
            # Se non esiste, creala
            os.makedirs(os.path.join(output_path,"intermediate_chartocrinput_dir"))
  #inputChartOCR_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/inputChartOCR"
  inputChartOCR_path=os.path.join(output_path,"intermediate_chartocrinput_dir")
  if not os.path.exists(os.path.join(output_path,"intermediate_chartocroutput_dir")):
            # Se non esiste, creala
            os.makedirs(os.path.join(output_path,"intermediate_chartocroutput_dir"))
  #outputChartOCR_path="/content/drive/MyDrive/ADSP_Project/Code/ChartPipeline/outputChartOCR"
  outputChartOCR_path=os.path.join(output_path,"intermediate_chartocroutput_dir")
  output_final_path=output_path

  runCapFigX(figCap_path,input_file_path,output_file_path)
  diz = copyCaptoChartOCR(output_file_path,inputChartOCR_path)
  runChartOCR(inputChartOCR_path,outputChartOCR_path)
  saving(outputChartOCR_path,output_final_path)

  return diz

In [ ]:
#main_chart(input_path="/content/ingresso",output_path="/content/uscita")

In [ ]:
def chartOCR_on_imgs(input_path="/content/drive/MyDrive/Code/ChartPipeline/inputChartOCR/copd-10-2109", output_path="/content/finalfinal"):
  os.chdir("/content/drive/MyDrive/ASDP Chart Extraction/DeepRule")
  !python "test_pipeline_new_old.py" --image_dir {input_path} --output_dir {output_path}
  if not os.path.exists(os.path.join(output_path,"final_output")):
    # Se non esiste, creala
    os.makedirs(os.path.join(output_path,"final_output"))
  saving(output_path,os.path.join(output_path,"final_output"),only_img=True)